In [ ]:
#@title Everything not mine is copyright 2020 Google LLC. Double-click here for full information.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Yann LeCun and Corinna Cortes hold the copyright of MNIST dataset,
# which is a derivative work from original NIST datasets. 
# MNIST dataset is made available under the terms of the 
# Creative Commons Attribution-Share Alike 3.0 license.

This Notebook is heavily modified from the MLCC programming project with single-digit images here:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/cc/exercises/multi-class_classification_with_MNIST.ipynb

**[Taken from the original MLCC "project" with single digit images. I will start with these images and then merge them together, pixel-array by pixel-array, to create a new train/test set of 100 double-digit images.]**

In [ ]:
# load some standard utilities.
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import random as rd
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

print("Loaded modules.")

In [ ]:
# load the dataset.
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
print("Loaded the train and test sets.")

In [ ]:
x_train_norm = x_train/255.
x_test_norm = x_test/255.

In [ ]:
x_train_norm.shape

## Create a deep neural net model

[Text from the original project for single digits.]

The `create_model` function defines the topography of the deep neural net, specifying the following:

* The number of [layers](https://developers.google.com/machine-learning/glossary/#layer) in the deep neural net.
* The number of [nodes](https://developers.google.com/machine-learning/glossary/#node) in each layer.
* Any [regularization](https://developers.google.com/machine-learning/glossary/#regularization) layers.

The `create_model` function also defines the [activation function](https://developers.google.com/machine-learning/glossary/#activation_function) of each layer.  The activation function of the output layer is [softmax](https://developers.google.com/machine-learning/glossary/#softmax), which will yield (100) different outputs for each example. Each of the (100) outputs provides the probability that the input example is a certain digit.

In [ ]:
# SET UP A DEEP NEURAL NET TO LEARN HANDRITTEN NUMBERS FROM 0 TO 99 #
# THIS CODE HERE IS ESSENTIALLY "THE ALGORITHM". 
# THERE'S NOT MUCH TO IT, ONCE EVERYTHING ELSE IS IN PLACE FOR IT!

def create_model(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.Dense(units=100, activation='softmax'))     

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model 

# THIS FUNCTION WILL TRAIN THE MODEL ON THE TRAINING SET #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):

    history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
    # Get a snapshot of the model's metrics after each round of training
    # to measure its progress.
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist    

## Invoke the previous functions to train on the train set and evaluate on the test set.

Run the following code cell to invoke the preceding functions and actually train the model on the training set. 

In [ ]:
# These variables are the adjustable "hyperparameters" of the model.
learning_rate = 0.001
epochs = 20
batch_size = 500
validation_split = 0.1

# CREATE A NEW NEURAL NETWORK ACCORDING TO SPECIFICATIONS.
my_model = create_model(learning_rate)

# TRAIN IT ON THE "NORMALIZED" TRAINING SET:
epochs, hist = train_model(my_model, x_train_norm, y_train, 
                           epochs, batch_size, validation_split)

# EVALUATE AGAINST THE TEST SET:
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=x_test_norm, y=y_test, batch_size=batch_size)

In [ ]:
# ABOVE 90% ACCURACY!
# Plot a graph of the 'accuracy' metric vs. epochs:
#plot_curve(epochs, hist, 'accuracy')

In [ ]:
# THIS FUNCTION TRANSLATES THE OUTPUT FROM THE NEURAL NETWORK
# INTO A "BEST GUESS" FOR EACH EXAMPLE BASED ON THE RELATIVE
# PROBABILITY IT ESTIMATES FOR EACH NUMBER 0 TO 9.

def getAnswers(how_many=10):  
    answers = pd.DataFrame(columns=['Answer','Guess','P(A)','P(G)'])  
    answers['Answer'] = y_test[:how_many]
    predicts = my_model.predict(x_test_norm).round(5)
    for j in range(0,how_many): # how_many is the number of examples to guess
        probs = predicts[j] # one row of 100 probabilities 
        maxr = max(probs)   # top probability
        for i in range(0,100):
            if probs[i] == maxr:
                answers.at[j,'Guess'] = i
                answers.at[j,'P(G)'] = maxr*100
            if i == answers['Answer'][j]:
                answers.at[j,'P(A)'] = probs[i]*100
    return answers
print("Loaded function getAnswers.")
print("Getting answers..." )

# LOAD UP ALL THE GUESSES (W/ PROBABILITES) FOR 
# EACH EXAMPLE IMAGE IN THE NORMALIZED TEST SET
answers = getAnswers(len(x_test_norm))
answers

In [ ]:
# FIND THE *WRONG* GUESSES!
wrongs = pd.DataFrame(index=answers.index,columns=['Mask'])
for row in answers.index:
    if answers['Answer'][row] == answers['Guess'][row]:
        wrongs['Mask'][row] = False
    else:
        wrongs['Mask'][row] = True

# LOCATE AND PRINT OUT THE LIST OF WRONGS       
a = answers.loc[wrongs['Mask']]
w = len(a) 
t = len(x_test_norm)
print(w,"wrong out of",t,"guesses.")
a

In [ ]:
# FIND OUT WHICH NUMBERS IT HAD THE MOST TROUBLE WITH
ww = []
for i in range(0,100):
    w = len((answers.loc[lambda a: a['Answer'] == i]).loc[wrongs['Mask']])
    ww += [w]
attempts = pd.DataFrame(ww)
attempts.sort_values(0).tail(5)

In [ ]:
g = answers['Guess'].loc[wrongs['Mask']]
a = answers['Answer'].loc[wrongs['Mask']]
x = rd.choice(g.index)  #RANDOMLY SELECT ONE OF THE WRONG GUESSES
pa = round(answers['P(A)'][x],2)
pg = round(answers['P(G)'][x],2)
print("Answer[",x,"] is",a[x],"with estimated probability",pa,"\n")
print("Guess[",x,"] was",g[x],"with estimated probability",pg,"\n")
plt.imshow(x_test_norm[x])

In [ ]:
#@title Everything not mine is copyright 2020 Google LLC. Double-click here for full information.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Yann LeCun and Corinna Cortes hold the copyright of MNIST dataset,
# which is a derivative work from original NIST datasets. 
# MNIST dataset is made available under the terms of the 
# Creative Commons Attribution-Share Alike 3.0 license.

This Notebook is heavily modified from the MLCC programming project with single-digit images here:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/cc/exercises/multi-class_classification_with_MNIST.ipynb

**[Taken from the original MLCC "project" with single digit images. I will start with these images and then merge them together, pixel-array by pixel-array, to create a new train/test set of 100 double-digit images.]**

In [ ]:
# load some standard utilities.
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import random as rd
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

print("Loaded modules.")

In [ ]:
# load the dataset.
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
print("Loaded the train and test sets.")

In [ ]:
x_train_norm = x_train/255.
x_test_norm = x_test/255.

In [ ]:
x_train_norm.shape

## Create a deep neural net model

[Text from the original project for single digits.]

The `create_model` function defines the topography of the deep neural net, specifying the following:

* The number of [layers](https://developers.google.com/machine-learning/glossary/#layer) in the deep neural net.
* The number of [nodes](https://developers.google.com/machine-learning/glossary/#node) in each layer.
* Any [regularization](https://developers.google.com/machine-learning/glossary/#regularization) layers.

The `create_model` function also defines the [activation function](https://developers.google.com/machine-learning/glossary/#activation_function) of each layer.  The activation function of the output layer is [softmax](https://developers.google.com/machine-learning/glossary/#softmax), which will yield (100) different outputs for each example. Each of the (100) outputs provides the probability that the input example is a certain digit.

In [ ]:
# SET UP A DEEP NEURAL NET TO LEARN HANDRITTEN NUMBERS FROM 0 TO 99 #
# THIS CODE HERE IS ESSENTIALLY "THE ALGORITHM". 
# THERE'S NOT MUCH TO IT, ONCE EVERYTHING ELSE IS IN PLACE FOR IT!

def create_model(learning_rate):
    """Create and compile a deep neural net."""  
    # Define the kind of model to use.
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))

    model.add(tf.keras.layers.Dense(units=100, activation='softmax'))     

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy']) 
    return model 

# THIS FUNCTION WILL TRAIN THE MODEL ON THE TRAINING SET #
def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.1):

    history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
    # Get a snapshot of the model's metrics after each round of training
    # to measure its progress.
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist    

## Invoke the previous functions to train on the train set and evaluate on the test set.

Run the following code cell to invoke the preceding functions and actually train the model on the training set. 

In [ ]:
# These variables are the adjustable "hyperparameters" of the model.
learning_rate = 0.001
epochs = 20
batch_size = 500
validation_split = 0.1

# CREATE A NEW NEURAL NETWORK ACCORDING TO SPECIFICATIONS.
my_model = create_model(learning_rate)

# TRAIN IT ON THE "NORMALIZED" TRAINING SET:
epochs, hist = train_model(my_model, x_train_norm, y_train, 
                           epochs, batch_size, validation_split)

# EVALUATE AGAINST THE TEST SET:
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x=x_test_norm, y=y_test, batch_size=batch_size)

In [ ]:
# ABOVE 90% ACCURACY!
# Plot a graph of the 'accuracy' metric vs. epochs:
#plot_curve(epochs, hist, 'accuracy')

In [ ]:
# THIS FUNCTION TRANSLATES THE OUTPUT FROM THE NEURAL NETWORK
# INTO A "BEST GUESS" FOR EACH EXAMPLE BASED ON THE RELATIVE
# PROBABILITY IT ESTIMATES FOR EACH NUMBER 0 TO 9.

def getAnswers(how_many=10):  
    answers = pd.DataFrame(columns=['Answer','Guess','P(A)','P(G)'])  
    answers['Answer'] = y_test[:how_many]
    predicts = my_model.predict(x_test_norm).round(5)
    for j in range(0,how_many): # how_many is the number of examples to guess
        probs = predicts[j] # one row of 100 probabilities 
        maxr = max(probs)   # top probability
        for i in range(0,100):
            if probs[i] == maxr:
                answers.at[j,'Guess'] = i
                answers.at[j,'P(G)'] = maxr*100
            if i == answers['Answer'][j]:
                answers.at[j,'P(A)'] = probs[i]*100
    return answers
print("Loaded function getAnswers.")
print("Getting answers..." )

# LOAD UP ALL THE GUESSES (W/ PROBABILITES) FOR 
# EACH EXAMPLE IMAGE IN THE NORMALIZED TEST SET
answers = getAnswers(len(x_test_norm))
answers

In [ ]:
# FIND THE *WRONG* GUESSES!
wrongs = pd.DataFrame(index=answers.index,columns=['Mask'])
for row in answers.index:
    if answers['Answer'][row] == answers['Guess'][row]:
        wrongs['Mask'][row] = False
    else:
        wrongs['Mask'][row] = True

# LOCATE AND PRINT OUT THE LIST OF WRONGS       
a = answers.loc[wrongs['Mask']]
w = len(a) 
t = len(x_test_norm)
print(w,"wrong out of",t,"guesses.")
a

In [ ]:
# FIND OUT WHICH NUMBERS IT HAD THE MOST TROUBLE WITH
ww = []
for i in range(0,100):
    w = len((answers.loc[lambda a: a['Answer'] == i]).loc[wrongs['Mask']])
    ww += [w]
attempts = pd.DataFrame(ww)
attempts.sort_values(0).tail(5)

In [ ]:
g = answers['Guess'].loc[wrongs['Mask']]
a = answers['Answer'].loc[wrongs['Mask']]
x = rd.choice(g.index)  #RANDOMLY SELECT ONE OF THE WRONG GUESSES
pa = round(answers['P(A)'][x],2)
pg = round(answers['P(G)'][x],2)
print("Answer[",x,"] is",a[x],"with estimated probability",pa,"\n")
print("Guess[",x,"] was",g[x],"with estimated probability",pg,"\n")
plt.imshow(x_test_norm[x])